In [ ]:
from qsextra import ExcitonicSystem, ChromophoreSystem
from qsextra.tools import spectral_function, unit_converter
from qsextra.spectroscopy import FeynmanDiagram, clspectroscopy, qspectroscopy
from qsextra.spectroscopy.postprocessing import postprocessing
import numpy as np
from scipy.linalg import toeplitz
import matplotlib.pyplot as plt

In [ ]:
epsilon = [1.55,1.46]    # [eV]
J = toeplitz([0., -0.01])    # [eV]
N = len(epsilon)

In [ ]:
esys = ExcitonicSystem(energies = epsilon,
                       dipole_moments = [1.] * N,
                       couplings = J,
                      )

In [ ]:
dt_fs = 0.1    # [fs]
t_final = 200.    # [fs]
t_list_plot = np.arange(0, t_final + dt_fs, dt_fs)    # [fs]
# Now convert to eV^-1
t_list = unit_converter(t_list_plot, initial_unit = 'fs', final_unit = 'eV-1')    # [eV-1]
dt = t_list[1] - t_list[0]    # [eV-1]
# For the quantum execution we reduce the time list in order to save computational time
times_plot = np.arange(0, t_final + dt_fs, 100 * dt_fs)    # [fs]
times = unit_converter(times_plot, initial_unit = 'fs', final_unit = 'eV-1')    # [eV-1]

Gamma = 59.08 * 10**(-3)

spec = FeynmanDiagram('a', times)

shots = 20000

# Exciton System

In [ ]:
signal_ex_q = qspectroscopy(esys,
                            spec,
                            shots=shots,
                            checkpoint=True,
                            coll_rates=Gamma/4,
                            dt=dt,
                           )
signal_ex = clspectroscopy(esys, spec, rates=Gamma/4)

In [ ]:
freq, spectrum_ex_q = postprocessing(spec,
                                     signal_ex_q,
                                     pad_extension = 3,
                                     RF_freq = 1.505,
                                    )
freq, spectrum_ex = postprocessing(spec,
                                   signal_ex,
                                   pad_extension = 3,
                                   RF_freq = 1.505,
                                  )

In [ ]:
plt.scatter(times_plot, signal_ex_q.real);
plt.scatter(times_plot, signal_ex_q.imag);
plt.plot(times_plot, signal_ex.real);
plt.plot(times_plot, signal_ex.imag);

In [ ]:
plt.plot(freq, spectrum_ex_q.real)
plt.plot(freq, spectrum_ex_q.imag)
plt.plot(freq, spectrum_ex.real, lw=0.4, c='k')
plt.plot(freq, spectrum_ex.imag, lw=0.4, c='k')
plt.xlim([1.3, 1.7]);

# Chromophore system

In [ ]:
sys = ChromophoreSystem(excitonic_system=esys)

In [ ]:
W = 1
frequencies_pseudomode = [0] * W
Gamma_list = [Gamma / W] * W
Omega_list = [0.1] * W
fr, sf = spectral_function(frequencies_pseudomode, Gamma_list, Omega_list)
plt.plot(fr, sf)

In [ ]:
levels_per_pseudomode = 2
sys.pseudomodes(frequencies_pseudomode = frequencies_pseudomode,
                levels_pseudomode = [levels_per_pseudomode] * len(frequencies_pseudomode),
                couplings_ep = np.sqrt(np.array(Gamma_list) * np.array(Omega_list) / 2).tolist(),
               )

In [ ]:
coll_rate = (2 * np.array(Omega_list)).tolist()

In [ ]:
signal_ch_q = qspectroscopy(sys,
                            spec,
                            shots=shots,
                            checkpoint=True,
                            coll_rates=coll_rate,
                            dt=dt,
                           )
signal_ch = clspectroscopy(sys, spec, rates=coll_rate)

In [ ]:
freq, spectrum_ch_q = postprocessing(spec,
                                     signal_ch_q,
                                     pad_extension = 3,
                                     RF_freq = 1.505,
                                    )
freq, spectrum_ch = postprocessing(spec,
                                   signal_ch,
                                   pad_extension = 3,
                                   RF_freq = 1.505,
                                  )

In [ ]:
plt.scatter(times_plot, signal_ch_q.real);
plt.scatter(times_plot, signal_ch_q.imag);
plt.plot(times_plot, signal_ch.real);
plt.plot(times_plot, signal_ch.imag);

In [ ]:
plt.plot(freq, spectrum_ch_q.real)
plt.plot(freq, spectrum_ch_q.imag)
plt.plot(freq, spectrum_ch.real, lw=0.4, c='k')
plt.plot(freq, spectrum_ch.imag, lw=0.4, c='k')
plt.xlim([1.3, 1.7]);

# Comparison

In [ ]:
plt.plot(freq, spectrum_ex_q.real, label = r'$\Omega \to \infty$')
plt.plot(freq, spectrum_ex.real, lw=0.4, c='dodgerblue', label = r'$\Omega \to \infty$ (exact)')
plt.plot(freq, spectrum_ch_q.real, label = r'$\Omega = 0.1$ eV')
plt.plot(freq, spectrum_ch.real, lw=0.4, c='coral', label = r'$\Omega = 0.1$ eV (exact)')
plt.legend()
plt.xlabel(r'$\omega$ (eV)')
plt.ylabel(r'amplitude (a.u.)');